# Inverse Optimization for DFS
_Applying Ghobadi and Mahmoudzadeh 2021_

In [ ]:
using JuMP
using Gurobi
using LinearAlgebra

In [ ]:
include("models/dflio.jl")

import .DflIo.Forward as Forward
import .DflIo.InverseDemand as IODemand
import .DflIo.InverseLinReg as IOLinReg

## Forward problem

In [ ]:
function forward_example_params()::Forward.Params
    enabled_flows = ones(Bool, (2, 2))
    enabled_flows[1, 2] = false

    return Forward.Params(
        n_paths=2, 
        n_commodities=2,
        capacities=[100, 100],
        design_costs=[100, 10],
        flow_costs=[10 10 ; 100 100],
        enabled_flows=enabled_flows
    )
end

function forward_example_demand()::Vector
    return [10, 6]
end

In [ ]:
forward_params = forward_example_params()
forward_demand = forward_example_demand()

forward_sol = Forward.create_and_solve_problem(forward_params, forward_demand)

In [ ]:
println(forward_sol.x_sol)
println(forward_sol.z_sol)

## Inverse problem (demand only)

In [ ]:
inverse_demand_model = IODemand.create_problem(forward_params, forward_sol)
print(inverse_demand_model)

In [ ]:
inverse_demand_solution = IODemand.solve_problem!(inverse_demand_model)

In [ ]:
print(inverse_demand_solution)

## Inverse problem (linear regression on demand)

In [ ]:
function inverse_linreg_example_params()
    weights = [1.5 2; 1 1]

    return IOLinReg.Params(weights=weights, forward_params=forward_example_params())
end

function inverse_linreg_example_featuress()
    return [[4, 2], [3, 1]]
end

In [ ]:
function predict_demand(params::IOLinReg.Params, linreg_features::Vector)
    return params.weights * linreg_features
end

function generate_solution_points(params::IOLinReg.Params, linreg_featuress)
    demandss = [predict_demand(params, features) for features in linreg_featuress]
    forward_sols = [Forward.create_and_solve_problem(params.forward_params, demands) for demands in demandss]

    return [IOLinReg.SolutionPoint(forward_sol, linreg_features) for (forward_sol, linreg_features) in zip(forward_sols, linreg_featuress)]
end

In [ ]:
linreg_params = inverse_linreg_example_params()
linreg_featuress = inverse_linreg_example_featuress()
solutions = generate_solution_points(linreg_params, linreg_featuress)

In [ ]:
linreg_model = IOLinReg.create_problem(linreg_params, solutions)
print(linreg_model)

In [ ]:
linreg_solution = IOLinReg.solve_problem!(linreg_model)
print(linreg_solution)